In [1]:
import git, os, sys
git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
os.chdir(f'{git_root}/src')
sys.path.append(os.path.abspath(os.path.join(f'{git_root}/src')))
print(f'Changed working directory to {os.getcwd()}')

Changed working directory to C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src


In [2]:
#getting dependencies
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn' (disables SettingWithCopyWarning)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import random
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
import itertools

# from fight_stat_helpers import *

In [ ]:
from sklearn.metrics import get_scorer_names

In [ ]:
print(get_scorer_names())

In [3]:
#scores a model
def model_score(dataframe, features, iloc_val = 3200, _max_iter = 2000, scoring='neg_log_loss', scaled=True):
    yyy=dataframe['result'].iloc[0:iloc_val]
    XXX=dataframe[features].iloc[0:iloc_val]
    XXXscaler = preprocessing.StandardScaler().fit(XXX)
    XXX_scaled = XXXscaler.transform(XXX) 
    X = XXX_scaled if scaled else XXX
    winPredictionModel=LogisticRegression(solver='lbfgs', max_iter=_max_iter, C=0.1, penalty='l2')#, fit_intercept=False)
    # find the cross val score with log loss
    return cross_val_score(winPredictionModel,X,yyy,cv=4,scoring=scoring).mean()
    
#CODE FOR THE GREEDY ALGORITHM FOR FEATURE SELECTION
def greedy(dataframe, features, subsetsize, iloc_val=3200, _max_iter = 2000, scaled=True, scoring='neg_log_loss', set_of_sets=False):
    if set_of_sets:
        s=set([tuple(feature) for feature in features])
    else:
        s=set(features)
    subsets=list(map(set, itertools.combinations(s, subsetsize))) #subsets of size (subsetsize)
    scores_dict = {}
    for subset in subsets:
        if set_of_sets:
            list_of_features = []
            for feature_set in subset:
                list_of_features.extend(list(feature_set))
        else:
            list_of_features = list(subset)
        scores_dict[tuple(subset)]=model_score(dataframe, list_of_features, iloc_val, _max_iter, scaled=scaled, scoring=scoring)
    max_key = max(scores_dict, key=scores_dict.get)
    max_score = scores_dict[max_key]
    print(f'best subset: {max_key}')
    print(f'with score {max_score}')
    best_feature_list = []
    if set_of_sets:
        for feature_set in max_key:
            best_feature_list.extend(list(feature_set))
    else:
        best_feature_list = list(max_key)
    return best_feature_list, max_score

def reductive_greedy(dataframe, starting_features=None, _max_iter=20000):
    # remove one at a time until it stops going up
    if starting_features is None:
        last_best_score = - 1000
        starting_features = dataframe.columns.tolist()
    else:
        last_best_score = model_score(dataframe, starting_features, _max_iter=_max_iter)
        print(f'Starting with features: {starting_features}')
        print(f'Current best score: {last_best_score}')
    # remove result from list 
    if 'result' in starting_features:
        starting_features.remove('result')
    current_best_score = model_score(dataframe, starting_features, _max_iter=_max_iter)
    features = starting_features.copy()
    while current_best_score > last_best_score:
        last_best_score = current_best_score
        new_features, current_best_score = greedy(dataframe, features, subsetsize=len(features)-1)
        features = new_features
        print(f'Current best score: {current_best_score}')
        print(f'Current best subset: {features}')
        
    print(f'No improvement found, stopping greedy search.')
    best_score = last_best_score
    print(f'Final best score: {best_score}')
    return features

# TODO show test set accuracy at each step as well. need to pass in test set and be careful to not use the test set in the model training.
def additive_greedy(dataframe, current_best_feature_set=[], search_doubles=False, _max_iter=5000, good_enough_acc=None):
    # now start adding remaining features back in one by one or in groups of two or three and see if we can improve the score
    if current_best_feature_set:
        current_model_score = model_score(dataframe, current_best_feature_set, _max_iter=_max_iter)
        current_model_accuracy = model_score(dataframe, current_best_feature_set, _max_iter=_max_iter, scoring='accuracy')
        print(f'Starting with current best feature set: {current_best_feature_set}')
        print(f'Current model score: {current_model_score}')
        print(f'Current model accuracy: {current_model_accuracy}')
    else:
        print('Starting with an empty feature set.')
        current_model_score = -1000 
        current_model_accuracy = 0.0
        
    while True:
        if good_enough_acc and current_model_accuracy >= good_enough_acc:
            print(f'Current model accuracy {current_model_accuracy} is above the threshold of {good_enough_acc}. Stopping search.')
            break
        unused_features = set(dataframe.columns) - set(current_best_feature_set) - set(['result'])
        # first try adding one feature 
        single_feature_scores = {}
        for feature in unused_features:
            new_feature_set = current_best_feature_set + [feature]
            new_model_score = model_score(dataframe, new_feature_set, _max_iter=_max_iter)
            single_feature_scores[tuple(new_feature_set)] = new_model_score
        best_single_feature_set = max(single_feature_scores, key=single_feature_scores.get)
        best_single_feature_score = single_feature_scores[best_single_feature_set]
        if best_single_feature_score > current_model_score:
            current_best_feature_set = list(best_single_feature_set)
            current_model_score = best_single_feature_score
            current_model_accuracy = model_score(dataframe, current_best_feature_set, _max_iter=_max_iter, scoring='accuracy')
            print(f'Added single feature: {best_single_feature_set[-1]}')
            print(f'New model score: {current_model_score}')
            print(f'New model accuracy: {current_model_accuracy}')
            continue
        else:
            print('No improvement found with single addition, trying pairs.')
        if not search_doubles:
            print('Stopping search for pairs (doubles flag set to false in inputs).')
            break
        double_feature_scores = {}
        for feature1, feature2 in itertools.combinations(unused_features, 2):
            new_feature_set = current_best_feature_set + [feature1, feature2]
            new_model_score = model_score(dataframe, new_feature_set, _max_iter=_max_iter)
            double_feature_scores[tuple(new_feature_set)] = new_model_score
        best_double_feature_set = max(double_feature_scores, key=double_feature_scores.get)
        best_double_feature_score = double_feature_scores[best_double_feature_set]
        if best_double_feature_score > current_model_score:
            current_best_feature_set = list(best_double_feature_set)
            current_model_score = best_double_feature_score
            current_model_accuracy = model_score(dataframe, current_best_feature_set, _max_iter=_max_iter, scoring='accuracy')

            print(f'Added double feature: {best_double_feature_set[-2]}, {best_double_feature_set[-1]}')
            print(f'New model score: {current_model_score}')
            print(f'New model accuracy: {current_model_accuracy}')

            continue
        else:
            print('No improvement found with double addition, stopping.')
            break

    print(f'Final best feature set: {current_best_feature_set}')
    return current_best_feature_set

In [4]:
ufc_fights_predictive_flattened_diffs_path = f'{git_root}/src/content/data/processed/ufc_fights_predictive_flattened_diffs2.csv'
ufc_fights_predictive_flattened_diffs = pd.read_csv(ufc_fights_predictive_flattened_diffs_path)
ufc_fights_predictive_flattened_diffs = ufc_fights_predictive_flattened_diffs[::-1]


@np.vectorize
def clean_method_for_winner_predictions(a):
    if (a == 'KO/TKO'):
        return 'KO/TKO'
    elif (a == 'SUB'):
        return 'SUB'
    elif ((a == 'U-DEC') or (a == 'M-DEC')):
        return 'DEC'
    # counting S-DEC as bullshit!
    else:
        return 'bullshit'
    
#fights where the method of victory is TKO/SUB/DEC (no split decision or DQ or Overturned or anything else like that)
ufc_fights_predictive_flattened_diffs['method'] = clean_method_for_winner_predictions(ufc_fights_predictive_flattened_diffs['method'])

draw_mask=ufc_fights_predictive_flattened_diffs['result'] != 'D'

# drop any rows with nan in any column
ufc_fights_predictive_flattened_diffs=ufc_fights_predictive_flattened_diffs[draw_mask] #&method_mask_winner] # TODO add method column for cleaning purposes

# method bullshit mask
method_mask_winner = ufc_fights_predictive_flattened_diffs['method'] != 'bullshit'
ufc_fights_predictive_flattened_diffs = ufc_fights_predictive_flattened_diffs[method_mask_winner]


In [5]:
# count the number of fights in each division
ufc_fights_predictive_flattened_diffs['division'].value_counts()

division
Lightweight              1231
Welterweight             1185
Middleweight              979
Featherweight             712
Heavyweight               682
Light Heavyweight         642
Bantamweight              627
Flyweight                 328
Women's Strawweight       288
Women's Flyweight         221
Women's Bantamweight      196
Open Weight                99
Catch Weight               66
Women's Featherweight      26
Super Heavyweight           1
Name: count, dtype: int64

In [6]:
# filter by fights only in lightweight or welterweight divisions
light_welter_mask = (ufc_fights_predictive_flattened_diffs['division'] == 'Lightweight') | (ufc_fights_predictive_flattened_diffs['division'] == 'Welterweight')
ufc_fights_predictive_flattened_diffs_light_welter = ufc_fights_predictive_flattened_diffs[light_welter_mask]
print(f'Number of fights in lightweight and welterweight divisions: {len(ufc_fights_predictive_flattened_diffs_light_welter)}')

Number of fights in lightweight and welterweight divisions: 2416


In [7]:
ufc_fights_predictive_flattened_diffs_light_welter = ufc_fights_predictive_flattened_diffs_light_welter.dropna(axis=0, how='any')
y = (ufc_fights_predictive_flattened_diffs_light_welter['result'] == 'W').values.astype(int)
# drop the 'fighter_result' column and opponent_result column as it is not needed for the model and also fighter and opponent columns
X = ufc_fights_predictive_flattened_diffs_light_welter.drop(columns=['fighter', 'opponent', 'method', 'division'])
# KEEP result in X as this is what model_score is expecting for better or for worse
X['result'] = (X['result'] == 'W').astype(int)  # Convert result to binary (1 for win, 0 for loss)
# TODO filter by division
# only take the first 4000 rows for training
# X = X.iloc[:4000, :]
# y = y[:4000]

# try this later
# ufc_fights_predictive_doubled_path = f'{git_root}/src/content/data/processed/ufc_fights_predictive_doubled.csv'
# ufc_fights_predictive_doubled = pd.read_csv(ufc_fights_predictive_doubled_path)

In [8]:
ufc_fights_predictive_flattened_diffs_light_welter.tail()

,fighter,opponent,result,method,division,age_diff,age_sum,height_diff,height_sum,reach_diff,reach_sum,all_wins_diff,all_wins_sum,l1y_wins_diff,l1y_wins_sum,l3y_wins_diff,l3y_wins_sum,l5y_wins_diff,l5y_wins_sum,all_wins_ko_diff,all_wins_ko_sum,l1y_wins_ko_diff,l1y_wins_ko_sum,l3y_wins_ko_diff,l3y_wins_ko_sum,l5y_wins_ko_diff,l5y_wins_ko_sum,all_wins_sub_diff,all_wins_sub_sum,l1y_wins_sub_diff,l1y_wins_sub_sum,l3y_wins_sub_diff,l3y_wins_sub_sum,l5y_wins_sub_diff,l5y_wins_sub_sum,all_wins_dec_diff,all_wins_dec_sum,l1y_wins_dec_diff,l1y_wins_dec_sum,l3y_wins_dec_diff,l3y_wins_dec_sum,l5y_wins_dec_diff,l5y_wins_dec_sum,all_losses_diff,all_losses_sum,l1y_losses_diff,l1y_losses_sum,l3y_losses_diff,l3y_losses_sum,l5y_losses_diff,l5y_losses_sum,all_losses_ko_diff,all_losses_ko_sum,l1y_losses_ko_diff,l1y_losses_ko_sum,l3y_losses_ko_diff,l3y_losses_ko_sum,l5y_losses_ko_diff,l5y_losses_ko_sum,all_losses_sub_diff,all_losses_sub_sum,l1y_losses_sub_diff,l1y_losses_sub_sum,l3y_losses_sub_diff,l3y_losses_sub_sum,l5y_losses_sub_diff,l5y_losses_sub_sum,all_losses_dec_diff,all_losses_dec_sum,l1y_losses_dec_diff,l1y_losses_dec_sum,l3y_losses_dec_diff,l3y_losses_dec_sum,l5y_losses_dec_diff,l5y_losses_dec_sum,all_num_fights_diff,all_num_fights_sum,l1y_num_fights_diff,l1y_num_fights_sum,l3y_num_fights_diff,l3y_num_fights_sum,l5y_num_fights_diff,l5y_num_fights_sum,all_inf_reversals_per_min_diff,all_inf_reversals_per_min_sum,l1y_inf_reversals_per_min_diff,l1y_inf_reversals_per_min_sum,l3y_inf_reversals_per_min_diff,l3y_inf_reversals_per_min_sum,l5y_inf_reversals_per_min_diff,l5y_inf_reversals_per_min_sum,all_abs_reversals_per_min_diff,all_abs_reversals_per_min_sum,l1y_abs_reversals_per_min_diff,l1y_abs_reversals_per_min_sum,l3y_abs_reversals_per_min_diff,l3y_abs_reversals_per_min_sum,l5y_abs_reversals_per_min_diff,l5y_abs_reversals_per_min_sum,all_inf_control_per_min_diff,all_inf_control_per_min_sum,l1y_inf_control_per_min_diff,l1y_inf_control_per_min_sum,l3y_inf_control_per_min_diff,l3y_inf_control_per_min_sum,l5y_inf_control_per_min_diff,l5y_inf_control_per_min_sum,all_abs_control_per_min_diff,all_abs_control_per_min_sum,l1y_abs_control_per_min_diff,l1y_abs_control_per_min_sum,l3y_abs_control_per_min_diff,l3y_abs_control_per_min_sum,l5y_abs_control_per_min_diff,l5y_abs_control_per_min_sum,all_inf_sub_attempts_per_min_diff,all_inf_sub_attempts_per_min_sum,l1y_inf_sub_attempts_per_min_diff,l1y_inf_sub_attempts_per_min_sum,l3y_inf_sub_attempts_per_min_diff,l3y_inf_sub_attempts_per_min_sum,l5y_inf_sub_attempts_per_min_diff,l5y_inf_sub_attempts_per_min_sum,all_abs_sub_attempts_per_min_diff,all_abs_sub_attempts_per_min_sum,l1y_abs_sub_attempts_per_min_diff,l1y_abs_sub_attempts_per_min_sum,l3y_abs_sub_attempts_per_min_diff,l3y_abs_sub_attempts_per_min_sum,l5y_abs_sub_attempts_per_min_diff,l5y_abs_sub_attempts_per_min_sum,all_inf_knockdowns_per_min_diff,all_inf_knockdowns_per_min_sum,l1y_inf_knockdowns_per_min_diff,l1y_inf_knockdowns_per_min_sum,l3y_inf_knockdowns_per_min_diff,l3y_inf_knockdowns_per_min_sum,l5y_inf_knockdowns_per_min_diff,l5y_inf_knockdowns_per_min_sum,all_abs_knockdowns_per_min_diff,all_abs_knockdowns_per_min_sum,l1y_abs_knockdowns_per_min_diff,l1y_abs_knockdowns_per_min_sum,l3y_abs_knockdowns_per_min_diff,l3y_abs_knockdowns_per_min_sum,l5y_abs_knockdowns_per_min_diff,l5y_abs_knockdowns_per_min_sum,all_inf_takedowns_landed_per_min_diff,all_inf_takedowns_landed_per_min_sum,all_inf_takedowns_attempts_per_min_diff,all_inf_takedowns_attempts_per_min_sum,all_inf_takedowns_accuracy_diff,all_inf_takedowns_accuracy_sum,l1y_inf_takedowns_landed_per_min_diff,l1y_inf_takedowns_landed_per_min_sum,l1y_inf_takedowns_attempts_per_min_diff,l1y_inf_takedowns_attempts_per_min_sum,l1y_inf_takedowns_accuracy_diff,l1y_inf_takedowns_accuracy_sum,l3y_inf_takedowns_landed_per_min_diff,l3y_inf_takedowns_landed_per_min_sum,l3y_inf_takedowns_attempts_per_min_diff,l3y_inf_takedowns_attempts_per_min_sum,l3y_inf_takedowns_accuracy_diff,l3y_inf_takedowns_accuracy_sum,l5y

In [ ]:
X.shape

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# NOTE A LARGER VALUE IS BETTER FOR NEG LOG LOSS

In [10]:
best_additive_features = additive_greedy(X_train, search_doubles=True, good_enough_acc=.75, _max_iter=50000) # 1215 too big for search doubles I think

Starting with an empty feature set.
Added single feature: l3y_wins_diff
New model score: -0.6713070653078634
New model accuracy: 0.572463768115942
Added single feature: age_diff
New model score: -0.6568462442445242
New model accuracy: 0.6141304347826086
Added single feature: l5y_offensive_grappling_score_diff
New model score: -0.6527509750912845
New model accuracy: 0.629830917874396
Added single feature: l5y_inf_ground_strikes_attempts_per_min_diff
New model score: -0.6502209085886169
New model accuracy: 0.6268115942028986
Added single feature: height_sum
New model score: -0.6477506266043386
New model accuracy: 0.6243961352657004
Added single feature: l5y_losses_ko_diff
New model score: -0.6459324379801303
New model accuracy: 0.6219806763285024
Added single feature: l5y_defensive_grappling_loss_diff
New model score: -0.6442962138765791
New model accuracy: 0.6286231884057971
Added single feature: l5y_inf_ground_strikes_accuracy_sum
New model score: -0.6429065699233277
New model accuracy

KeyboardInterrupt: 

In [32]:
# result without squares
best_additive_features = [
    'l3y_wins_diff',
    'age_diff',
    'l5y_offensive_grappling_score_diff',
    'l5y_inf_ground_strikes_attempts_per_min_diff',
    'height_sum',
    'l5y_losses_ko_diff',
    'l5y_defensive_grappling_loss_diff',
    'l5y_inf_ground_strikes_accuracy_sum',
    'all_inf_takedowns_attempts_per_min_diff',
    'l1y_inf_clinch_strikes_accuracy_sum',
    'l1y_wins_sub_sum',
    'l1y_inf_total_strikes_accuracy_sum',
    'l5y_abs_ground_strikes_landed_per_min_diff',
    'l1y_abs_ground_strikes_attempts_per_min_diff',
    'all_abs_ground_strikes_attempts_per_min_diff',
    'l1y_abs_control_per_min_diff',
    'l1y_abs_clinch_strikes_accuracy_diff',
    'l1y_inf_takedowns_attempts_per_min_diff',
    'l3y_abs_body_strikes_landed_per_min_sum',
    'all_abs_leg_strikes_landed_per_min_sum',
    'l1y_losses_ko_diff',
    'l1y_abs_ground_strikes_accuracy_sum',
    'l5y_abs_ground_strikes_accuracy_sum',
    'l1y_abs_sub_attempts_per_min_sum',
    'all_abs_sub_attempts_per_min_sum',
    'reach_diff',
    'all_inf_reversals_per_min_diff',
    'l3y_inf_reversals_per_min_diff',
    'l1y_abs_takedowns_landed_per_min_sum',
    'l1y_abs_control_per_min_sum',
    'all_inf_head_strikes_attempts_per_min_sum',
    'l5y_abs_distance_strikes_accuracy_sum',
    'l1y_inf_control_per_min_sum',
    'l3y_inf_ground_strikes_attempts_per_min_sum',
    'l3y_inf_body_strikes_accuracy_sum',
    'all_inf_ground_strikes_landed_per_min_sum',
    'l3y_losses_sum',
    'l1y_inf_takedowns_accuracy_diff',
    'l3y_inf_leg_strikes_accuracy_diff',
    'l5y_inf_total_strikes_attempts_per_min_diff',
    'all_inf_total_strikes_landed_per_min_diff',
    'l3y_inf_head_strikes_landed_per_min_diff',
    'all_inf_clinch_strikes_landed_per_min_diff',
    'l3y_inf_clinch_strikes_attempts_per_min_diff',
    'l3y_inf_ground_strikes_accuracy_diff',
    'l3y_abs_distance_strikes_accuracy_sum',
    'all_abs_body_strikes_attempts_per_min_diff',
    'l5y_abs_body_strikes_landed_per_min_diff',
    'l1y_losses_dec_sum',
    'l1y_inf_sig_strikes_accuracy_diff',
    'l1y_inf_clinch_strikes_attempts_per_min_diff',
    'l5y_inf_head_strikes_attempts_per_min_diff',
    'l3y_inf_clinch_strikes_landed_per_min_diff',
    'l5y_num_fights_diff',
    'l1y_inf_ground_strikes_landed_per_min_diff',
    'l5y_inf_knockdowns_per_min_diff',
    'l1y_inf_knockdowns_per_min_diff',
    'l5y_abs_body_strikes_attempts_per_min_diff',
    'l3y_inf_total_strikes_attempts_per_min_diff',
    'all_inf_body_strikes_landed_per_min_diff',
    'l3y_abs_knockdowns_per_min_sum',
    'l3y_abs_sub_attempts_per_min_diff',
    'l5y_abs_sub_attempts_per_min_diff',
    'l3y_abs_ground_strikes_landed_per_min_diff',
    'l3y_abs_ground_strikes_attempts_per_min_diff',
    'l1y_abs_body_strikes_accuracy_diff',
    'l1y_inf_knockdowns_per_min_sum',
    'all_inf_knockdowns_per_min_diff',
    'all_inf_reversals_per_min_sum',
    'l5y_inf_ground_strikes_landed_per_min_diff',
    'all_abs_distance_strikes_attempts_per_min_diff',
    'all_inf_head_strikes_accuracy_diff',
    'l3y_abs_clinch_strikes_accuracy_diff',
    'l1y_abs_body_strikes_landed_per_min_diff',
    'l3y_inf_body_strikes_accuracy_diff',
    'all_abs_clinch_strikes_accuracy_diff',
    'all_inf_body_strikes_accuracy_diff',
    'all_abs_reversals_per_min_sum',
    'l5y_abs_reversals_per_min_sum',
    'l1y_abs_distance_strikes_accuracy_sum',
    'l1y_abs_sig_strikes_accuracy_sum',
    'all_abs_total_strikes_accuracy_sum',
    'l5y_abs_sig_strikes_accuracy_sum',
    'all_inf_ground_strikes_accuracy_sum',
    'l5y_inf_clinch_strikes_landed_per_min_diff',
    'l3y_inf_head_strikes_attempts_per_min_diff',
    'all_wins_dec_diff',
    'all_inf_head_strikes_landed_per_min_sum',
    'l1y_inf_total_strikes_accuracy_diff',
    'l5y_inf_ground_strikes_landed_per_min_sum',
    'l3y_inf_ground_strikes_landed_per_min_sum',
    'l5y_defensive_grappling_loss_sum',
    'l1y_abs_head_strikes_landed_per_min_diff',
    'l1y_abs_head_strikes_accuracy_diff',
    'all_abs_head_strikes_landed_per_min_diff',
    'l1y_inf_distance_strikes_attempts_per_min_diff',
    'all_abs_head_strikes_attempts_per_min_diff',
    'all_inf_distance_strikes_landed_per_min_diff',
    'l1y_inf_clinch_strikes_accuracy_diff',
    'l1y_inf_clinch_strikes_landed_per_min_diff',
    'l1y_losses_dec_diff',
    'l5y_abs_clinch_strikes_landed_per_min_diff',
    'all_inf_head_strikes_attempts_per_min_diff',
    'l1y_inf_sub_attempts_per_min_diff',
    'l3y_inf_sub_attempts_per_min_diff',
    'l3y_abs_head_strikes_landed_per_min_diff',
    'l1y_inf_sig_strikes_attempts_per_min_diff',
    'l1y_abs_total_strikes_attempts_per_min_diff', 
    'l1y_abs_total_strikes_landed_per_min_diff',
    'all_inf_leg_strikes_accuracy_diff',
    'l3y_abs_body_strikes_accuracy_diff',
    'l3y_inf_sig_strikes_accuracy_diff',
    'all_abs_body_strikes_landed_per_min_diff',
    'all_inf_ground_strikes_accuracy_diff',
    'all_abs_leg_strikes_accuracy_diff',
    'all_abs_total_strikes_landed_per_min_diff',
    'all_abs_clinch_strikes_landed_per_min_diff',
]

In [39]:
# result without squares
best_additive_features_no_sum = [
    'l3y_wins_diff',
    'age_diff',
    'age_sum',
    'l5y_offensive_grappling_score_diff',
    'l5y_inf_ground_strikes_attempts_per_min_diff',
    'l5y_losses_ko_diff',
    'l5y_defensive_grappling_loss_diff',
    'all_inf_takedowns_attempts_per_min_diff',
    'l5y_abs_ground_strikes_landed_per_min_diff',
    'l1y_abs_ground_strikes_attempts_per_min_diff',
    'all_abs_ground_strikes_attempts_per_min_diff',
    'l1y_abs_control_per_min_diff',
    'l1y_abs_clinch_strikes_accuracy_diff',
    'l1y_inf_takedowns_attempts_per_min_diff',
    'l1y_losses_ko_diff',
    'reach_diff',
    'all_inf_reversals_per_min_diff',
    'l3y_inf_reversals_per_min_diff',
    'l3y_inf_ground_strikes_attempts_per_min_sum',
    'all_inf_ground_strikes_landed_per_min_sum',
    'l1y_inf_takedowns_accuracy_diff',
    'l3y_inf_leg_strikes_accuracy_diff',
    'l5y_inf_total_strikes_attempts_per_min_diff',
    'all_inf_total_strikes_landed_per_min_diff',
    'l3y_inf_head_strikes_landed_per_min_diff',
    'all_inf_clinch_strikes_landed_per_min_diff',
    'l3y_inf_clinch_strikes_attempts_per_min_diff',
    'l3y_inf_ground_strikes_accuracy_diff',
    'all_abs_body_strikes_attempts_per_min_diff',
    'l5y_abs_body_strikes_landed_per_min_diff',
    'l1y_inf_sig_strikes_accuracy_diff',
    'l1y_inf_clinch_strikes_attempts_per_min_diff',
    'l5y_inf_head_strikes_attempts_per_min_diff',
    'l3y_inf_clinch_strikes_landed_per_min_diff',
    'l5y_num_fights_diff',
    'l1y_inf_ground_strikes_landed_per_min_diff',
    'l5y_inf_knockdowns_per_min_diff',
    'l1y_inf_knockdowns_per_min_diff',
    'l5y_abs_body_strikes_attempts_per_min_diff',
    'l3y_inf_total_strikes_attempts_per_min_diff',
    'all_inf_body_strikes_landed_per_min_diff',
    'l3y_abs_sub_attempts_per_min_diff',
    'l5y_abs_sub_attempts_per_min_diff',
    'l3y_abs_ground_strikes_landed_per_min_diff',
    'l3y_abs_ground_strikes_attempts_per_min_diff',
    'l1y_abs_body_strikes_accuracy_diff',
    'l1y_inf_knockdowns_per_min_sum',
    'all_inf_knockdowns_per_min_diff',
    'all_inf_reversals_per_min_sum',
    'l5y_inf_ground_strikes_landed_per_min_diff',
    'all_abs_distance_strikes_attempts_per_min_diff',
    'all_inf_head_strikes_accuracy_diff',
    'l3y_abs_clinch_strikes_accuracy_diff',
    'l1y_abs_body_strikes_landed_per_min_diff',
    'l3y_inf_body_strikes_accuracy_diff',
    'all_abs_clinch_strikes_accuracy_diff',
    'all_inf_body_strikes_accuracy_diff',
    'l5y_inf_clinch_strikes_landed_per_min_diff',
    'l3y_inf_head_strikes_attempts_per_min_diff',
    'all_wins_dec_diff',
    'l1y_inf_total_strikes_accuracy_diff',
    'l1y_abs_head_strikes_landed_per_min_diff',
    'l1y_abs_head_strikes_accuracy_diff',
    'all_abs_head_strikes_landed_per_min_diff',
    'l1y_inf_distance_strikes_attempts_per_min_diff',
    'all_abs_head_strikes_attempts_per_min_diff',
    'all_inf_distance_strikes_landed_per_min_diff',
    'l1y_inf_clinch_strikes_accuracy_diff',
    'l1y_inf_clinch_strikes_landed_per_min_diff',
    'l1y_losses_dec_diff',
    'l5y_abs_clinch_strikes_landed_per_min_diff',
    'all_inf_head_strikes_attempts_per_min_diff',
    'l1y_inf_sub_attempts_per_min_diff',
    'l3y_inf_sub_attempts_per_min_diff',
    'l3y_abs_head_strikes_landed_per_min_diff',
    'l1y_inf_sig_strikes_attempts_per_min_diff',
    'l1y_abs_total_strikes_attempts_per_min_diff', 
    'l1y_abs_total_strikes_landed_per_min_diff',
    'all_inf_leg_strikes_accuracy_diff',
    'l3y_abs_body_strikes_accuracy_diff',
    'l3y_inf_sig_strikes_accuracy_diff',
    'all_abs_body_strikes_landed_per_min_diff',
    'all_inf_ground_strikes_accuracy_diff',
    'all_abs_leg_strikes_accuracy_diff',
    'all_abs_total_strikes_landed_per_min_diff',
    'all_abs_clinch_strikes_landed_per_min_diff',
]

In [40]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# see how the new features do on the test set we already made
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[best_additive_features_no_sum])
# train the model with the best features

best_model = LogisticRegression(solver='lbfgs', max_iter=20000)#, fit_intercept=False)
best_model.fit(X_train_scaled, y_train)

# evaluate the model on the training set
train_score = best_model.score(X_train_scaled, y_train)

print(f'Train set accuracy: {train_score}')

# evaluate the model on the test set
X_test_scaled = scaler.transform(X_test[best_additive_features_no_sum])
test_score = best_model.score(X_test_scaled, y_test)
print(f'Test set accuracy: {test_score} \n')


Train set accuracy: 0.677536231884058
Test set accuracy: 0.6231884057971014 



In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# JUST ADDING AGE SUM HELPS THE TEST SET A TON

# see how the new features do on the test set we already made
# train the model with the best features

best_model = LogisticRegression(solver='lbfgs', max_iter=20000)#, fit_intercept=False)
best_model.fit(X_train[best_additive_features_no_sum], y_train)

# evaluate the model on the training set
train_score = best_model.score(X_train[best_additive_features_no_sum], y_train)

print(f'Train set accuracy: {train_score}')

# evaluate the model on the test set
test_score = best_model.score(X_test[best_additive_features_no_sum], y_test)
print(f'Test set accuracy: {test_score} \n')


Train set accuracy: 0.6733091787439613
Test set accuracy: 0.6352657004830918 



In [23]:
X_test_scaled.shape

(414, 117)

In [ ]:
# result when we had squares
# training accuracy was over 70 and test was at 60
best_additive_features = ['l3y_wins_diff',
'age_sq_diff',
'l5y_inf_takedowns_landed_per_min_diff',
'l1y_inf_distance_strikes_landed_per_min_sq_sum',
'l5y_losses_ko_sq_diff',
'l5y_inf_ground_strikes_attempts_per_min_diff',
'l3y_inf_leg_strikes_landed_per_min_sum',
'l5y_defensive_grappling_loss_diff',
'l1y_abs_clinch_strikes_accuracy_sq_diff',
'l5y_abs_head_strikes_accuracy_sq_diff',
'all_inf_takedowns_attempts_per_min_diff',
'l1y_inf_takedowns_attempts_per_min_sq_diff',
'l1y_abs_clinch_strikes_landed_per_min_sq_diff',
'height_sq_sum',
'all_inf_reversals_per_min_sq_diff',
'all_wins_dec_sq_sum',
'l3y_losses_dec_sq_sum',
'all_inf_clinch_strikes_attempts_per_min_sum',
'l1y_inf_head_strikes_attempts_per_min_sq_sum',
'l1y_abs_body_strikes_attempts_per_min_sq_sum',
'l5y_wins_ko_diff',
'l1y_abs_sub_attempts_per_min_sum',
'l1y_abs_knockdowns_per_min_sq_sum',
'l3y_abs_ground_strikes_landed_per_min_sq_sum',
'l1y_inf_control_per_min_sq_sum',
'all_inf_total_strikes_landed_per_min_sum',
'l5y_abs_takedowns_accuracy_sq_sum',
'l5y_inf_ground_strikes_accuracy_sq_sum',
'l3y_inf_clinch_strikes_accuracy_sum',
'l1y_abs_clinch_strikes_attempts_per_min_diff',
'all_abs_body_strikes_attempts_per_min_sq_diff',
'l5y_inf_head_strikes_attempts_per_min_sq_diff',
'l1y_inf_sig_strikes_landed_per_min_sq_diff',
'l3y_inf_distance_strikes_landed_per_min_diff',
'l3y_inf_ground_strikes_accuracy_sq_diff',
'l1y_abs_takedowns_accuracy_sq_sum',
'l1y_abs_takedowns_accuracy_sum',
'l3y_wins_sub_sum',
'l3y_abs_body_strikes_attempts_per_min_sq_diff',
'l1y_inf_takedowns_accuracy_diff',
'reach_diff',
'l5y_inf_sig_strikes_attempts_per_min_sq_diff',
'l1y_wins_ko_diff',
'l3y_wins_ko_diff',
'all_abs_total_strikes_attempts_per_min_sq_diff',
'l1y_abs_head_strikes_landed_per_min_sq_diff',
'l5y_abs_head_strikes_landed_per_min_sq_diff',
'l1y_abs_body_strikes_landed_per_min_diff',
'l1y_inf_clinch_strikes_attempts_per_min_sq_diff',
'l1y_wins_dec_sq_diff',
'l3y_abs_head_strikes_accuracy_sq_diff',
'l3y_losses_sub_sum',
'l5y_defensive_grappling_loss_sq_sum',
'l5y_num_fights_sq_sum',
'l5y_losses_dec_sum',
'l1y_inf_head_strikes_landed_per_min_sum',
'l3y_inf_clinch_strikes_landed_per_min_diff',
'l1y_abs_head_strikes_accuracy_sq_diff',
'l3y_losses_dec_sum',
'l3y_inf_leg_strikes_accuracy_sq_diff',
'l1y_abs_control_per_min_sum',
'l3y_abs_control_per_min_sq_sum',
'l5y_inf_head_strikes_accuracy_sq_sum',
'l3y_inf_sig_strikes_accuracy_sq_sum',
'all_inf_head_strikes_accuracy_sq_diff',
'l3y_inf_sig_strikes_accuracy_sq_diff',
'l3y_inf_body_strikes_landed_per_min_sq_diff',
'l1y_inf_body_strikes_accuracy_sq_sum',
'all_inf_clinch_strikes_landed_per_min_sum',
'l3y_inf_clinch_strikes_attempts_per_min_diff',
'all_abs_control_per_min_sq_sum',
'l1y_inf_distance_strikes_attempts_per_min_diff',
'l3y_losses_sq_diff',
'l1y_wins_ko_sum',
'all_abs_takedowns_attempts_per_min_diff',
'l1y_abs_takedowns_attempts_per_min_sq_diff',
'l1y_abs_takedowns_attempts_per_min_diff',
'l1y_abs_control_per_min_diff',
'l3y_inf_clinch_strikes_landed_per_min_sq_diff',
'all_inf_clinch_strikes_landed_per_min_diff',
'l3y_inf_distance_strikes_accuracy_sum',
'all_inf_distance_strikes_landed_per_min_diff',
'l1y_abs_total_strikes_landed_per_min_diff',
'l5y_abs_knockdowns_per_min_sq_diff',
'l1y_abs_leg_strikes_attempts_per_min_sum',
'l5y_abs_leg_strikes_landed_per_min_sq_sum',
'l3y_inf_clinch_strikes_attempts_per_min_sq_sum',
'all_inf_clinch_strikes_landed_per_min_sq_diff',
'l3y_inf_body_strikes_accuracy_sq_diff',
'l5y_inf_body_strikes_accuracy_diff',
'l1y_wins_ko_sq_sum',
'l1y_inf_total_strikes_attempts_per_min_sq_sum',
'l3y_abs_ground_strikes_accuracy_sum',
'all_abs_ground_strikes_accuracy_sum',
'l1y_inf_reversals_per_min_sq_diff',
'l3y_losses_sub_sq_sum',
'l3y_defensive_grappling_loss_sq_sum',
'all_inf_clinch_strikes_attempts_per_min_diff',
'l1y_inf_clinch_strikes_attempts_per_min_sq_sum',
'l5y_wins_sq_sum',
'l5y_losses_sq_sum',
'all_losses_ko_sum',
'all_wins_sub_sq_sum',
'l1y_abs_control_per_min_sq_diff',
'l5y_inf_ground_strikes_attempts_per_min_sq_sum',
'l5y_wins_sub_sum',
'l5y_losses_sum',
'all_losses_dec_diff',
'l1y_losses_ko_sq_diff',
'l1y_losses_ko_diff',
'all_losses_sq_diff',
'l3y_inf_body_strikes_landed_per_min_diff',
'l5y_inf_ground_strikes_accuracy_sum',
'l5y_abs_ground_strikes_accuracy_sq_sum',
'all_abs_ground_strikes_accuracy_sq_sum',
'all_wins_sum',
'l5y_inf_clinch_strikes_landed_per_min_sq_sum',
'l1y_inf_clinch_strikes_landed_per_min_sq_sum',
'l3y_abs_total_strikes_attempts_per_min_sq_diff',
'all_inf_distance_strikes_accuracy_sq_diff',
'l5y_abs_ground_strikes_landed_per_min_sum',
'l1y_abs_ground_strikes_attempts_per_min_sum',
'l3y_inf_sig_strikes_attempts_per_min_sq_diff',
'l5y_inf_clinch_strikes_landed_per_min_diff',
'l5y_inf_reversals_per_min_sq_diff',
'all_inf_total_strikes_landed_per_min_sq_sum',
'l5y_inf_ground_strikes_attempts_per_min_sq_diff',
'l3y_inf_ground_strikes_attempts_per_min_sq_sum',
'l3y_abs_ground_strikes_landed_per_min_sum',
'l1y_inf_sig_strikes_attempts_per_min_sq_diff',
'all_num_fights_sq_sum',
'l1y_inf_head_strikes_landed_per_min_sq_sum',
'l3y_abs_leg_strikes_attempts_per_min_sq_diff',
'all_abs_leg_strikes_attempts_per_min_diff',
'all_inf_clinch_strikes_landed_per_min_sq_sum',
'all_inf_ground_strikes_attempts_per_min_sq_sum',
'all_abs_leg_strikes_landed_per_min_sq_sum',
'l3y_inf_ground_strikes_landed_per_min_sq_diff',
]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# see how the new features do on the test set we already made
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[best_additive_features])
# train the model with the best features

best_model = LogisticRegression(solver='lbfgs', max_iter=20000, C=0.1, penalty='l2')#, fit_intercept=False)
best_model.fit(X_train_scaled, y_train)

# evaluate the model on the training set
train_score = best_model.score(X_train_scaled, y_train)

print(f'Train set accuracy: {train_score}')

# evaluate the model on the test set
X_test_scaled = scaler.transform(X_test[best_additive_features])
test_score = best_model.score(X_test_scaled, y_test)
print(f'Test set accuracy: {test_score} \n')


In [ ]:
len(best_additive_features)

In [ ]:
for feature in best_additive_features:
    print(feature)

In [ ]:
# remove one at a time until it stops going up
best_reductive_subset = reductive_greedy(X_train, starting_features=best_additive_features)

In [ ]:
best_model_score = model_score(X_train, best_reductive_subset, _max_iter=50000)
print(f'Best model score: {best_model_score}')
print(f'Probability to observe data given the model: {np.exp(best_model_score)}')
accuracy = model_score(X_train, best_additive_features, _max_iter=50000, scoring='accuracy')
print(f'Accuracy of the model: {accuracy}')

In [ ]:
# train model on training set with best features 
# use scaler to scale the features

# split X and y into training and test sets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

test_size = 0.3
print(f'Splitting data into training and test sets with test size: {test_size}')
print(f'size of X_train: {X.shape[0]* (1-test_size)}, size of X_test: {X.shape[0]* test_size}')

for random_state in range(10):
    print(f'Running with random state: {random_state}')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train[best_additive_features])
    # train the model with the best features

    best_model = LogisticRegression(solver='lbfgs', max_iter=20000, C=0.1, penalty='l2')#, fit_intercept=False)
    best_model.fit(X_train_scaled, y_train)

    # evaluate the model on the training set
    train_score = best_model.score(X_train_scaled, y_train)

    print(f'Train set accuracy: {train_score}')

    # evaluate the model on the test set
    X_test_scaled = scaler.transform(X_test[best_additive_features])
    test_score = best_model.score(X_test_scaled, y_test)

    print(f'Test set accuracy: {test_score} \n')

In [ ]:
# train model on training set with best features 
# use scaler to scale the features

# split X and y into training and test sets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

test_size = 0.3
print(f'Splitting data into training and test sets with test size: {test_size}')
print(f'size of X_train: {X.shape[0]* (1-test_size)}, size of X_test: {X.shape[0]* test_size}')

for random_state in range(10,20):
    print(f'Running with random state: {random_state}')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train[best_additive_features])
    # train the model with the best features

    best_model = LogisticRegression(solver='lbfgs', max_iter=20000, C=0.1, penalty='l2')#, fit_intercept=False)
    best_model.fit(X_train_scaled, y_train)

    # evaluate the model on the training set
    train_score = best_model.score(X_train_scaled, y_train)

    print(f'Train set accuracy: {train_score}')

    # evaluate the model on the test set
    X_test_scaled = scaler.transform(X_test[best_additive_features])
    test_score = best_model.score(X_test_scaled, y_test)

    print(f'Test set accuracy: {test_score} \n')

In [ ]:
# do another split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=12)

In [ ]:
best_model_score = model_score(X_train, best_reductive_subset, _max_iter=50000)
print(f'Best model score: {best_model_score}')
print(f'Probability to observe data given the model: {np.exp(best_model_score)}')
accuracy = model_score(X_train, best_additive_features, _max_iter=50000, scoring='accuracy')
print(f'Accuracy of the model: {accuracy}')

In [ ]:
# try adding more features to the model (still no doubles) not stopping at 65%
updated_additive_features = additive_greedy(X_train, current_best_feature_set=best_additive_features, search_doubles=False, _max_iter=50000)

In [ ]:
# see how the new features do on the test set we already made
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[updated_additive_features])
# train the model with the best features

best_model = LogisticRegression(solver='lbfgs', max_iter=20000, C=0.002, penalty='l2')#, fit_intercept=False)
best_model.fit(X_train_scaled, y_train)

# evaluate the model on the training set
train_score = best_model.score(X_train_scaled, y_train)

print(f'Train set accuracy: {train_score}')

# evaluate the model on the test set
X_test_scaled = scaler.transform(X_test[updated_additive_features])
test_score = best_model.score(X_test_scaled, y_test)
print(f'Test set accuracy: {test_score} \n')


In [ ]:
# train model on training set with best features 
# use scaler to scale the features

# split X and y into training and test sets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

test_size = 0.3
print(f'Splitting data into training and test sets with test size: {test_size}')
print(f'size of X_train: {X.shape[0]* (1-test_size)}, size of X_test: {X.shape[0]* test_size}')

for random_state in range(30, 40):
    print(f'Running with random state: {random_state}')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train[updated_additive_features])
    # train the model with the best features

    best_model = LogisticRegression(solver='lbfgs', max_iter=20000, C=0.1, penalty='l2')#, fit_intercept=False)
    best_model.fit(X_train_scaled, y_train)

    # evaluate the model on the training set
    train_score = best_model.score(X_train_scaled, y_train)

    print(f'Train set accuracy: {train_score}')

    # evaluate the model on the test set
    X_test_scaled = scaler.transform(X_test[updated_additive_features])
    test_score = best_model.score(X_test_scaled, y_test)

    print(f'Test set accuracy: {test_score} \n')

In [ ]:
# find the best regularization params
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],  # l1 is not supported by lbfgs solver
    'solver': ['lbfgs'],
    'max_iter': [1000, 2000, 5000, 10000]
}
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)
print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_}')
best_model = grid_search.best_estimator_
# evaluate the best model on the test set
X_test_scaled = scaler.transform(X_test[best_additive_features])
test_score = best_model.score(X_test_scaled, y_test)

# so use c= 0.1
# TODO maybe try this again on a partial higher scoring feature set


In [ ]:
# WOW THIS IS AMAZING
print('Best additive features:')
print('-------------------------')
print(f'Number of best additive features: {len(best_additive_features)}')
for col in best_additive_features:
    print(col)

In [ ]:
amazing_feature_set = [
    'age_diff',
    'l1y_inf_clinch_strikes_attempts_per_min_diff',
    'l5y_inf_head_strikes_accuracy_diff',
    'all_wins_diff',
    'l5y_abs_total_strikes_accuracy_diff',
    'all_abs_leg_strikes_accuracy_diff',
    'l2y_abs_leg_strikes_accuracy_diff',
    'l5y_inf_body_strikes_accuracy_diff',
    'l1y_abs_reversals_per_min_diff',
    'l2y_abs_clinch_strikes_accuracy_diff',
    'l1y_inf_leg_strikes_accuracy_diff',
    'all_wins_dec_diff',
    'all_inf_takedowns_accuracy_diff',
    'l1y_wins_ko_diff',
    'l1y_inf_ground_strikes_accuracy_diff',
    'l3y_inf_ground_strikes_accuracy_diff',
    'l1y_abs_distance_strikes_attempts_per_min_diff',
    'l1y_abs_body_strikes_landed_per_min_diff',
    'l1y_inf_distance_strikes_landed_per_min_diff',
    'l3y_abs_takedowns_accuracy_diff',
    'all_abs_takedowns_accuracy_diff',
    'l3y_inf_distance_strikes_accuracy_diff',
    'l1y_offensive_standing_striking_score_diff',
    'l2y_wins_sub_diff',
    'all_abs_body_strikes_accuracy_diff',
    'l5y_abs_body_strikes_accuracy_diff',
    'l2y_defensive_grappling_loss_diff',
    'l3y_abs_body_strikes_accuracy_diff',
    'all_inf_knockdowns_per_min_diff',
    'l3y_defensive_grappling_loss_diff',
    'l2y_inf_head_strikes_attempts_per_min_diff',
    'l3y_inf_head_strikes_attempts_per_min_diff',
    'l1y_abs_total_strikes_landed_per_min_diff',
    'l3y_abs_leg_strikes_accuracy_diff',
    'l5y_abs_sig_strikes_accuracy_diff',
    'l2y_inf_sig_strikes_accuracy_diff',
    'l2y_inf_head_strikes_landed_per_min_diff',
    'l1y_abs_ground_strikes_accuracy_diff',
    'l1y_inf_sig_strikes_accuracy_diff',
    'l3y_inf_reversals_per_min_diff',
    'all_inf_reversals_per_min_diff',
    'l2y_wins_ko_diff',
    'l1y_inf_head_strikes_landed_per_min_diff',
    'l2y_inf_ground_strikes_accuracy_diff',
    'l3y_abs_takedowns_attempts_per_min_diff',
    'all_abs_takedowns_landed_per_min_diff',
    'l1y_abs_takedowns_attempts_per_min_diff',
    'l5y_inf_takedowns_attempts_per_min_diff',
    'l3y_inf_takedowns_attempts_per_min_diff',
    'l5y_inf_knockdowns_per_min_diff',
    'l3y_num_fights_diff',
    'l1y_inf_clinch_strikes_landed_per_min_diff',
    'l1y_inf_clinch_strikes_accuracy_diff',
    'l3y_inf_distance_strikes_attempts_per_min_diff',
    'l2y_num_fights_diff',
    'l5y_inf_ground_strikes_accuracy_diff',
    'all_abs_head_strikes_landed_per_min_diff',
    'l5y_abs_head_strikes_landed_per_min_diff',
    'l1y_abs_sig_strikes_attempts_per_min_diff',
    'l3y_abs_head_strikes_attempts_per_min_diff',
    'l5y_abs_takedowns_landed_per_min_diff',
    'l5y_inf_leg_strikes_attempts_per_min_diff',
    'l3y_inf_leg_strikes_landed_per_min_diff',
    'l1y_inf_distance_strikes_attempts_per_min_diff',
    'l1y_wins_dec_diff',
    'l3y_losses_dec_diff',
    'l2y_losses_ko_diff',
    'l5y_losses_ko_diff',
    'all_inf_leg_strikes_attempts_per_min_diff',
    'l5y_inf_leg_strikes_accuracy_diff',
    'l3y_inf_clinch_strikes_landed_per_min_diff',
    'l5y_inf_clinch_strikes_landed_per_min_diff',
    'l1y_abs_clinch_strikes_landed_per_min_diff',
    'l3y_abs_clinch_strikes_landed_per_min_diff',
    'l1y_abs_clinch_strikes_accuracy_diff',
    'l1y_abs_ground_strikes_attempts_per_min_diff',
    'l5y_abs_clinch_strikes_landed_per_min_diff',
]

In [ ]:
# try removing features again 
best_reductive_subset = reductive_greedy(X, starting_features=amazing_feature_set)

In [ ]:
# probability of observing data given the model
best_model_score = model_score(X, best_reductive_subset, _max_iter=5000)
print(f'Probability to observe data given the model: {np.exp(best_model_score)}')

In [ ]:
X.shape

In [ ]:
# get accuracy of model
accuracy = model_score(X, amazing_feature_set, _max_iter=5000, scoring='accuracy')
print(f'Accuracy of the model: {accuracy}')

# WHY IS THE SCORE DIFFERENT THAN FROM ABOVE???

In [ ]:
def ufc_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    # TODO rescale the features to zero mean and unit variance
    
    # return [fighter_score_diff(fighter1,fighter2,day1, 4),
    #         fighter_score_diff(fighter1,fighter2,day1, 9),
    #         fighter_score_diff(fighter1,fighter2,day1, 15),
    #         fight_math_diff(fighter1,fighter2,day1, 1),
    #         fight_math_diff(fighter1,fighter2,day1, 6),
    #         L5Y_sub_wins(fighter1,day1)-L5Y_sub_wins(fighter2,day2),
    #         L5Y_losses(fighter1,day1)-L5Y_losses(fighter2,day2),
    #         L5Y_ko_losses(fighter1,day1)-L5Y_ko_losses(fighter2,day2),
    #         fighter_age(fighter1,day1)-fighter_age(fighter2,day2),
    #         avg_count('total_strikes_landed',fighter1,'abs',day1)-avg_count('total_strikes_landed',fighter2,'abs',day2),
    #         avg_count('head_strikes_landed',fighter1,'abs',day1)-avg_count('head_strikes_landed',fighter2,'abs',day2),
    #         avg_count('ground_strikes_landed',fighter1,'inf',day1)-avg_count('ground_strikes_landed',fighter2,'inf',day2),
    #         avg_count('takedowns_attempts',fighter1,'inf',day1)-avg_count('takedowns_attempts',fighter2,'inf',day2),
    #         avg_count('head_strikes_landed',fighter1,'inf',day1)-avg_count('head_strikes_landed',fighter2,'inf',day2),
    #     ]
    return [
    # 'fighter_age_diff',
    #     'height_diff',
    #     'reach_diff',
    #     'fighter_L5Y_wins_diff_2',
    #     'fighter_L5Y_losses_diff_2',
    #     'fighter_ko_losses_diff_2',
    #     'fighter_L5Y_ko_wins_diff_2',
    #     'fighter_L2Y_ko_losses_diff_2',
        fighter_age(fighter1, day1) - fighter_age(fighter2, day2),
        fighter_height(fighter1) - fighter_height(fighter2),
        fighter_reach(fighter1) - fighter_reach(fighter2),
        L5Y_wins(fighter1, day1) - L5Y_wins(fighter2, day2),
        L5Y_losses(fighter1, day1) - L5Y_losses(fighter2, day2),
        ko_losses(fighter1, day1) - ko_losses(fighter2, day2),
        L5Y_ko_wins(fighter1, day1) - L5Y_ko_wins(fighter2, day2),
        L2Y_ko_losses(fighter1, day1) - L2Y_ko_losses(fighter2, day2),
        #     'fighter_L5Y_ko_losses_diff_2',
    #     'fighter_sub_wins_diff_2',
    #     'fighter_L2Y_sub_losses_diff_2',
    #     'fighter_inf_knockdowns_avg_diff_2',
        L2Y_ko_losses(fighter1, day1) - L2Y_ko_losses(fighter2, day2),
        sub_wins(fighter1, day1) - sub_wins(fighter2, day2),
        L2Y_sub_losses(fighter1, day1) - L2Y_sub_losses(fighter2, day2),
        avg_count('knockdowns', fighter1, 'inf', day1) - avg_count('knockdowns', fighter2, 'inf', day2),
    
    #     'fighter_inf_distance_strikes_landed_avg_diff_2',
        avg_count('distance_strikes_landed', fighter1, 'inf', day1) - avg_count('distance_strikes_landed', fighter2, 'inf', day2),
    #     'fighter_inf_ground_strikes_landed_avg_diff_2',
        avg_count('ground_strikes_landed', fighter1, 'inf', day1) - avg_count('ground_strikes_landed', fighter2, 'inf', day2),
    #     'fighter_inf_distance_strikes_attempts_avg_diff_2',
        avg_count('distance_strikes_attempts', fighter1, 'inf', day1) - avg_count('distance_strikes_attempts', fighter2, 'inf', day2),
        
        
        #     'fighter_inf_head_strikes_attempts_avg_diff_2',
        avg_count('head_strikes_attempts', fighter1, 'inf', day1) - avg_count('head_strikes_attempts', fighter2, 'inf', day2),
    #     'fighter_inf_takedowns_attempts_avg_diff_2',
        avg_count('takedowns_attempts', fighter1, 'inf', day1) - avg_count('takedowns_attempts', fighter2, 'inf', day2),
    #     'fighter_abs_reversals_avg_diff_2',
        # avg_count('reversals', fighter1, 'abs', day1) - avg_count('reversals', fighter2, 'abs', day2),
    #     'fighter_abs_clinch_strikes_landed_avg_diff_2',
        avg_count('clinch_strikes_landed', fighter1, 'abs', day1) - avg_count('clinch_strikes_landed', fighter2, 'abs', day2),
    #     'fighter_abs_distance_strikes_landed_avg_diff_2',
        avg_count('distance_strikes_landed', fighter1, 'abs', day1) - avg_count('distance_strikes_landed', fighter2, 'abs', day2),
        
            #     'fighter_abs_leg_strikes_landed_avg_diff_2',
        avg_count('leg_strikes_landed', fighter1, 'abs', day1) - avg_count('leg_strikes_landed', fighter2, 'abs', day2),
    #     'fighter_abs_takedowns_attempts_avg_diff_2',
        avg_count('takedowns_attempts', fighter1, 'abs', day1) - avg_count('takedowns_attempts', fighter2, 'abs', day2),
    #     'fighter_abs_distance_strikes_attempts_avg_diff_2',
        avg_count('distance_strikes_attempts', fighter1, 'abs', day1) - avg_count('distance_strikes_attempts', fighter2, 'abs', day2),
    #     'fighter_abs_head_strikes_attempts_avg_diff_2',
        avg_count('head_strikes_attempts', fighter1, 'abs', day1) - avg_count('head_strikes_attempts', fighter2, 'abs', day2),
    #     'fighter_abs_total_strikes_attempts_avg_diff_2',
        avg_count('total_strikes_attempts', fighter1, 'abs', day1) - avg_count('total_strikes_attempts', fighter2, 'abs', day2),
                #     '6-fight_math',
    #     '9-fighter_score_diff',
    #     '4-fighter_score_diff',
        fight_math_diff(fighter1, fighter2, day1, 6),
        fighter_score_diff(fighter1, fighter2, day1, 9),
        fighter_score_diff(fighter1, fighter2, day1, 4),
    ]
    

In [ ]:
dataframe = ufc_fights_winner
iloc_val = 3200
_max_iter = 2000
scoring='neg_log_loss'
scaled=True

yyy=dataframe['result'].iloc[0:iloc_val]
XXX=dataframe[current_best_feature_set].iloc[0:iloc_val]
XXXscaler = preprocessing.StandardScaler().fit(XXX)
XXX_scaled = XXXscaler.transform(XXX) 
X = XXX_scaled if scaled else XXX
winPredictionModel=LogisticRegression(solver='lbfgs', max_iter=_max_iter)#, fit_intercept=False)
# find the cross val score with log loss
cross_val_score(winPredictionModel,X,yyy,cv=4,scoring=scoring).mean()

In [ ]:
tup = ufc_prediction_tuple('Cody Garbrandt', 'Pedro Munhoz', 'July 5, 2023', 'July 5, 2023')

In [ ]:
tup_scaled = XXXscaler.transform(np.array(tup).reshape(1, -1))

In [ ]:
tup_scaled[0]